# Load Util File with TARE Model Functions

In [1]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Print verbsose output variable
print_debug = False

# import from cmu-tare-model package
from config import PROJECT_ROOT
import pandas as pd

# Set columns in display
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns') # Reset options to default

# Set rows in display
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows') # Reset options to default

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set_theme(style="darkgrid")

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Storing Result Outputs in output_results folder
relative_path = os.path.join("cmu_tare_model", "output_results")
output_folder_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: {PROJECT_ROOT}
Result outputs will be exported here: {output_folder_path}

""")

Project root directory: c:\users\14128\research\cmu-tare-model

-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results




# Simulate Residential Energy Consumption using NREL End-Use Savings Shapes

In [2]:
from cmu_tare_model.energy_consumption_and_metadata.user_input_geographic_filter import *
from cmu_tare_model.energy_consumption_and_metadata.process_euss_data import *

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)

file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id'

# Filter for occupied homes
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'
    location_id = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)    
    print(f"You chose to filter for: {input_state}")

    location_id = str(input_state)
    print(f"Location ID is: {location_id}")


    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
        location_id = str(input_state)
        print(f"Location ID is: {location_id}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")

        location_id = input_cityFilter.replace(', ', '_').strip()
        print(f"Location ID is: {location_id}")

        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

print(f"""
====================================================================================================================================================================
BASELINE (Measure Package 0)
====================================================================================================================================================================
DATAFRAME: df_euss_am_baseline

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, State, or City

Additional details data can be found in the official End-Use Load Profiles/Savings Shapes documentation.
Our methodology is detailed in the process_euss_data.py file and its associated imports.
      
DATAFRAME: df_euss_am_baseline
      
{df_euss_am_baseline}
""")

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv
You chose to filter for: PA
Location ID is: PA

To accurately characterize load profile, it is recommended to select subsets of data with >= 1000 models (~240,000 representative dwelling units).

The following cities (number of models also shown) are available for this state:

in.city
Not in a census Place      7035
In another census Place    5851
PA, Philadelphia           1631
PA, Pittsburgh              329
PA, Erie                    105
PA, Allentown               104
PA, Reading                  92
PA, Bethlehem                85
PA, Scranton                 80
PA, Harrisburg               63
PA, Lancaster                58
PA, Levittown                58
PA, Altoona                  56
PA, York                     56
PA, Wilkes-Barre             4

In [3]:
# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(
    df_baseline = df_euss_am_baseline
    )

print(f"""
In addition to the housing type and occupancy filters, the data has been filtered for fuel and technology
Please see the energy_consumption_and_metadata folder for more details on data processing.

df_enduse_refactored() will display the number of representative homes after filtering for fuel and tech.
      
DATAFRAME: df_euss_am_baseline_home
      
{df_euss_am_baseline_home}
""")

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
Calculated total heating consumption
Calculated total waterHeating consumption
Calculated total clothesDrying consumption
Calculated total cooking consumption

Creating data quality flags for all categories

--- Processing heating ---
Values in base_heating_fuel (top 5):
base_heating_fuel
Natural Gas    8594
Fuel Oil       2922
Electricity    2722
Other Fuel      789
Propane         615
Name: count, dtype: int64
  heating: Found 798 homes (5.1%) with invalid fuel types
  Invalid fuel types (top 5):
base_heating_fuel
Other Fuel    789
Name: count, dtype: int64
Values in heating_type (top 5):
heating_type
Natural Gas Fuel Furnace               5686
Fuel Oil Fuel Furnace                  1855
Natural Gas Fuel Wall/Floor Furnace    1437
Natural Gas Fuel Boiler                1301
Electricity Bas

## Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data (Factors)

In [4]:
print(f"""
====================================================================================================================================================================
PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption
====================================================================================================================================================================

def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:

Creating dataframe to store annual energy consumption calculations ...

""")

from cmu_tare_model.energy_consumption_and_metadata.project_future_energy_consumption import *


PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption

def project_future_consumption(
    df: pd.DataFrame, 
    menu_mp: int,
    base_year: int = 2024
) -> Tuple[pd.DataFrame, pd.DataFrame]:

Creating dataframe to store annual energy consumption calculations ...


Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\projections\aeo_projections_2022_2050.xlsx


In [5]:
# Make a copy of the dataframe
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_scenario_consumption = df_euss_am_baseline_home.copy()

# Project Future Energy Consumption
df_euss_am_baseline_home, df_baseline_scenario_consumption = project_future_consumption(
    df=df_euss_am_baseline_home,
    menu_mp=menu_mp,
    base_year=2024
    )

print(f"""
====================================================================================================================================================================
      
DATAFRAME: df_euss_am_baseline_home

{df_euss_am_baseline_home}      

DATAFRAME: df_baseline_scenario_consumption
      
{df_baseline_scenario_consumption}
""")


Determining valid homes for calculations:
Baseline calculation for heating:
  - 12266 homes have valid data
  - 3385 homes have invalid data (values will be NaN)
Baseline calculation for waterHeating:
  - 14999 homes have valid data
  - 652 homes have invalid data (values will be NaN)
Baseline calculation for clothesDrying:
  - 14743 homes have valid data
  - 908 homes have invalid data (values will be NaN)
Baseline calculation for cooking:
  - 7867 homes have valid data
  - 7784 homes have invalid data (values will be NaN)

===== Processing HEATING =====
Projecting consumption for 12266 out of 15651 homes
Projecting Future Energy Consumption (Baseline Equipment): heating

===== Processing WATERHEATING =====
Projecting consumption for 14999 out of 15651 homes
Projecting Future Energy Consumption (Baseline Equipment): waterHeating

===== Processing CLOTHESDRYING =====
Projecting consumption for 14743 out of 15651 homes
Projecting Future Energy Consumption (Baseline Equipment): clothesD

In [6]:
print_debug = True
if print_debug:
    from cmu_tare_model.utils.create_sample_df import create_sample_df
    # Create a sample dataframe
    # def create_sample_df(
    #     df: pd.DataFrame,
    #     include_groups: Optional[List[str]] = None,
    #     categories: Optional[List[str]] = None,
    #     scenarios: Optional[List[str]] = None,
    #     metrics: Optional[List[str]] = None,
    #     mp_number: int = 8,
    #     regex_patterns: Optional[Union[str, List[str]]] = None  # New parameter
    # ) -> pd.DataFrame:

    # # Set columns in display
    # pd.set_option('display.max_columns', None)
    # pd.reset_option('display.max_columns') # Reset options to default

    # # Set rows in display
    # pd.set_option('display.max_rows', None)
    # pd.reset_option('display.max_rows') # Reset options to default

    df_sample_heating = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_heating', 'valid_tech_heating', 'include_heating']
    )
    print(df_sample_heating)


    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['waterHeating'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_waterHeating', 'valid_tech_waterHeating', 'include_waterHeating']
    )
    print(df_sample_waterHeating)


    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['clothesDrying'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_clothesDrying', 'valid_tech_clothesDrying', 'include_clothesDrying']
    )
    print(df_sample_clothesDrying)


    df_sample_cooking = create_sample_df(
        df=df_euss_am_baseline_home,
        include_groups=['base_equipment', 'consumption',],
        categories=['cooking'],
        # scenarios=['in.annual_energy_kwh'],
        # metrics=['in.annual_energy_kwh'],
        # mp_number=menu_mp,
        regex_patterns=['valid_fuel_cooking', 'valid_tech_cooking', 'include_cooking']
    )
    print(df_sample_cooking)


        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                  0.0000

# Public Perspective: Monetized Marginal Damages from Emissions

## Fossil Fuels: Climate and Health-Related Pollutants

In [7]:
print(f"""
====================================================================================================================================================================
DETERMINE CLIMATE CHANGE AND PUBLIC HEALTH IMPACTS
====================================================================================================================================================================
Detailed documentation of the methods used to calculate the climate and public health impacts can be
found in the public_impacts folder of the cmu-tare-model package. All monetary values are in 2023
inflation-adjusted dollars ($USD2023).
- data_processing sub-folder: contains the data processing scripts used for lookup dictionaries
- calculations sub-folder: contains the calculations for emissions from fossul fuel appliances
- Main folder contains the scripts for calculating lifetime climate and public health impacts as
  as well as climate NPV, health NPV, and combined public NPV.
=====================================================================================================================================================================
""")

from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import *
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import *


DETERMINE CLIMATE CHANGE AND PUBLIC HEALTH IMPACTS
Detailed documentation of the methods used to calculate the climate and public health impacts can be
found in the public_impacts folder of the cmu-tare-model package. All monetary values are in 2023
inflation-adjusted dollars ($USD2023).
- data_processing sub-folder: contains the data processing scripts used for lookup dictionaries
- calculations sub-folder: contains the calculations for emissions from fossul fuel appliances
- Main folder contains the scripts for calculating lifetime climate and public health impacts as
  as well as climate NPV, health NPV, and combined public NPV.

Fuel sources in df_grid_mix: {'Renewable', 'Nuclear', 'Natural Gas', 'Oil', 'Coal'}
Fuel sources in df_grid_emis_factors: {'Nuclear', 'Natural Gas', 'Renewables', 'Oil', 'Coal'}
Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\inflation_data\bls_cpiu_2005-2023.xlsx
    year

### Baseline Marginal Damages: WHOLE-HOME

In [8]:
print(f"""
====================================================================================================================================================================
CLIMATE CHANGE IMPACTS: Baseline Scenario
====================================================================================================================================================================
Modified usage pattern - keeping df_euss_am_baseline_home and detailed results separate

def calculate_lifetime_climate_impacts(
    df: pd.DataFrame,
    menu_mp: int,
    policy_scenario: str,
    base_year: int = 2024,
    df_baseline_damages: Optional[pd.DataFrame] = None,
    verbose: bool = False    
) -> Tuple[pd.DataFrame, pd.DataFrame]:

""")

# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_damages_climate = df_euss_am_baseline_home.copy()

# Modified usage pattern - keeping df_euss_am_baseline_home as a single DataFrame
# while keeping detailed results separate
df_euss_am_baseline_home, df_baseline_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_baseline_home,
    menu_mp=0,  # baseline
    policy_scenario='No Inflation Reduction Act',
    verbose=True  # Add this parameter
)


CLIMATE CHANGE IMPACTS: Baseline Scenario
Modified usage pattern - keeping df_euss_am_baseline_home and detailed results separate

def calculate_lifetime_climate_impacts(
    df: pd.DataFrame,
    menu_mp: int,
    policy_scenario: str,
    base_year: int = 2024,
    df_baseline_damages: Optional[pd.DataFrame] = None,
    verbose: bool = False    
) -> Tuple[pd.DataFrame, pd.DataFrame]:


-- Scenario: Baseline -- 
              scenario_prefix: 'baseline_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Climate Emissions and Damages from 2024 to 2039 for heating
Calculating Climate Emissions and Damages from 2024 to 2036 for waterHeating
Calculating Climate Emissions and Damages from 2024 to 203

In [9]:
print(f"""
====================================================================================================================================================================

df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}

df_baseline_damages_climate: DataFrame containing the baseline scenario data with climate damages      
{df_baseline_damages_climate}

""")

# Climate Change Impacts: Baseline Scenario
if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_euss_am_baseline_home.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)

    # df_baseline_damages_climate
    print(f"Shape of df_baseline_damages_climate: {df_baseline_damages_climate.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_baseline_damages_climate.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)



df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic 

In [10]:
print(f"""
====================================================================================================================================================================
PUBLIC HEALTH IMPACTS: Baseline Scenario
====================================================================================================================================================================
Modified usage pattern - keeping df_euss_am_baseline_home and detailed results separate

def calculate_lifetime_health_impacts(
    df: pd.DataFrame,
    menu_mp: int,
    policy_scenario: str,
    base_year: int = 2024,
    df_baseline_damages: Optional[pd.DataFrame] = None,
    debug: bool = False,
    verbose: bool = False
) -> Tuple[pd.DataFrame, pd.DataFrame]:

""")

# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_damages_health = df_euss_am_baseline_home.copy()

df_euss_am_baseline_home, df_baseline_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_baseline_home,
    menu_mp=0,  # baseline
    policy_scenario='No Inflation Reduction Act',
    debug=True,
    verbose=True  # Add this parameter
)


PUBLIC HEALTH IMPACTS: Baseline Scenario
Modified usage pattern - keeping df_euss_am_baseline_home and detailed results separate

def calculate_lifetime_health_impacts(
    df: pd.DataFrame,
    menu_mp: int,
    policy_scenario: str,
    base_year: int = 2024,
    df_baseline_damages: Optional[pd.DataFrame] = None,
    debug: bool = False,
    verbose: bool = False
) -> Tuple[pd.DataFrame, pd.DataFrame]:



HEALTH IMPACT DATA COVERAGE ANALYSIS

Analyzed 67 counties across 1 states

FOSSIL FUEL HEALTH IMPACT COVERAGE:
----------------------------------------
Model: AP2, CR Function: acs
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 counties (0.0%)
  - Missing data: 0/67 counties (0.0%)
  - Total coverage: 67/67 counties (100.0%)
Model: AP2, CR Function: h6c
  - County-level data: 67/67 counties (100.0%)
  - State-level fallback: 0/67 counties (0.0%)
  - Missing data: 0/67 counties (0.0%)
  - Total coverage: 67/67 counties (100.0%)
Model: EASIUR, CR Func

In [11]:
print(f"""
====================================================================================================================================================================

df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}
      
df_baseline_damages_health: DataFrame containing the baseline scenario data with health damages
{df_baseline_damages_health}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_euss_am_baseline_home.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)

    # df_baseline_damages_health
    print(f"Shape of df_baseline_damages_health: {df_baseline_damages_health.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_baseline_damages_health.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)



df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic 

## Private Perspective: Annual Energy Costs

### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [12]:
print("""
=====================================================================================================================================================================
LIFETIME FUEL COSTS: Baseline Scenario
=====================================================================================================================================================================
STEPS
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
      
""")
from cmu_tare_model.private_impact.calculate_lifetime_fuel_costs import *

# Make copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_baseline_home = df_euss_am_baseline_home.copy()
df_baseline_fuel_costs = df_euss_am_baseline_home.copy()

# def calculate_lifetime_climate_impacts(df: pd.DataFrame, menu_mp: int, policy_scenario: str, df_baseline_damages: Optional[pd.DataFrame] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
# Returns df_main, df_detailed
df_euss_am_baseline_home, df_baseline_fuel_costs = calculate_lifetime_fuel_costs(
    df=df_euss_am_baseline_home,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act'
    )


LIFETIME FUEL COSTS: Baseline Scenario
STEPS
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
      

-- Scenario: Baseline -- 
              scenario_prefix: 'baseline_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Fuel Costs from 2024 to 2039 for heating
Calculating Fuel Costs from 2024 to 2036 for waterHeating
Calculating Fuel Costs from 2024 to 2037 for clothesDrying
Calculating Fuel Costs from 2024 to 2039 for cooking


In [17]:
print(f"""
====================================================================================================================================================================

df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}
      
df_baseline_fuel_costs: DataFrame containing the baseline scenario data with fuel costs
{df_baseline_fuel_costs}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "fuelCost"
    fuelCost_columns = [col for col in df_euss_am_baseline_home.columns if "fuelCost" in col]
    print("\nColumns containing 'fuelCost':")
    print(fuelCost_columns)

    # df_baseline_fuel_costs
    print(f"Shape of df_baseline_fuel_costs: {df_baseline_fuel_costs.shape}")

    # Print columns that contain the word "damages"
    fuelCost_columns = [col for col in df_baseline_fuel_costs.columns if "fuelCost" in col]
    print("\nColumns containing 'fuelCost':")
    print(fuelCost_columns)



df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic 

# Model Runtime

In [14]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 1 minutes and 26 seconds to execute.
